
<img src="../../img/ods_stickers.jpg">

## <center> [mlcourse.ai](https://mlcourse.ai) – открытый курс OpenDataScience по машинному обучению 
    
Автор материала: Юрий Кашницкий (@yorko в Slack ODS). Материал распространяется на условиях лицензии [Creative Commons CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/). Можно использовать в любых целях (редактировать, поправлять и брать за основу), кроме коммерческих, но с обязательным упоминанием автора материала.

# <center>Домашнее задание 6 (демо). Линейная регрессия, Lasso и RF-регрессия в задаче по определению качества вина</center>
<img src='../../img/wine_quality.jpg' width=30%>

**Заполните пропущенный код и ответьте на вопросы в [онлайн-форме](https://docs.google.com/forms/d/1gsNxgkd0VqidZp4lh9mnCQnJw3b0IFR1C4WBES86J40).**

In [2]:
# отключим всякие предупреждения Anaconda
import warnings

warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Lasso, LassoCV, LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import (GridSearchCV, cross_val_score,
                                     train_test_split)
from sklearn.preprocessing import StandardScaler

**Будем работать с набором данных по качеству белого вина (репозиторий UCI).**
**Загружаем данные.**

In [3]:
data = pd.read_csv('winequality-white.csv', sep=';')

In [4]:
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4898 entries, 0 to 4897
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         4898 non-null   float64
 1   volatile acidity      4898 non-null   float64
 2   citric acid           4898 non-null   float64
 3   residual sugar        4898 non-null   float64
 4   chlorides             4898 non-null   float64
 5   free sulfur dioxide   4898 non-null   float64
 6   total sulfur dioxide  4898 non-null   float64
 7   density               4898 non-null   float64
 8   pH                    4898 non-null   float64
 9   sulphates             4898 non-null   float64
 10  alcohol               4898 non-null   float64
 11  quality               4898 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 459.3 KB


**Отделите целевой признак, разделите обучающую выборку в отношении 7:3 (30% - под оставленную выборку, пусть random_state=17) и отмасштабируйте данные с помощью StandardScaler.**

In [37]:
y = data['quality']
x = data.drop('quality',axis=1)

X_train, X_holdout, y_train, y_holdout = train_test_split(x,y,random_state=17,train_size=0.7) # Ваш код здесь
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train) # Ваш код здесь
X_holdout_scaled = scaler.transform(X_holdout) # Ваш код здесь

## Линейная регрессия

**Обучите простую линейную регрессию.**

In [38]:
linreg = LinearRegression()# Ваш код здесь
linreg.fit(X_train_scaled,y_train); # Ваш код здесь
linreg_predict = linreg.predict(X_holdout_scaled)
linreg_predict_train = linreg.predict(X_train_scaled)

**<font color='red'>Вопрос 1:</font> Каковы среднеквадратичные ошибки линейной регрессии на обучающей и отложенной выборках?**

In [39]:
print("Mean squared error (train): %.3f" % mean_squared_error(y_pred=linreg_predict_train,y_true=y_train))
print("Mean squared error (test): %.3f" % mean_squared_error(y_pred=linreg_predict,y_true=y_holdout))

Mean squared error (train): 0.558
Mean squared error (test): 0.584


**Посмотрите на коэффициенты модели и отранжируйте признаки по влиянию на качество вина (учтите, что большие по модулю отрицательные значения коэффициентов тоже говорят о сильном влиянии). Создайте для этого новый небольшой DataFrame.**<br>
**<font color='red'>Вопрос 2:</font> Какой признак линейная регрессия считает наиболее сильно влияющим на качество вина?**

In [40]:
linreg_coef = pd.DataFrame(linreg.coef_, X_train.columns)
linreg_coef.sort_values(by=0,key=abs,ascending=False)

,0
density,-0.665720
residual sugar,0.538164
volatile acidity,-0.192260
pH,0.150036
alcohol,0.129533
fixed acidity,0.097822
sulphates,0.062053
free sulfur dioxide,0.042180
total sulfur dioxide,0.014304
chlorides,0.008127


## Lasso-регрессия

**Обучите Lasso-регрессию с небольшим коэффициентом $\alpha = 0.01$ (слабая регуляризация). Пусть опять random_state=17.**

In [42]:
lasso1 = Lasso(alpha=0.01,random_state=17) # Ваш код здесь
lasso1.fit(X_train_scaled,y_train); # Ваш код здесь 

**Посмотрите на коэффициенты модели и отранжируйте признаки по влиянию на качество вина. Какой признак "отвалился" первым, то есть наименее важен для объяснения целевого признака в модели Lasso?**

In [43]:
lasso1_coef = pd.DataFrame(lasso1.coef_,X_train.columns) # Ваш код здесь
lasso1_coef.sort_values(by=0,key=abs,ascending=False) # Ваш код здесь

,0
alcohol,0.322425
residual sugar,0.256363
density,-0.235492
volatile acidity,-0.188479
pH,0.067277
free sulfur dioxide,0.043088
sulphates,0.029722
chlorides,-0.002747
fixed acidity,-0.000000
citric acid,-0.000000


**Теперь определите лучшее значение $\alpha$ в процессе кросс-валидации 5-кратной кросс-валидации. Используйте LassoCV и random_state=17.**

In [49]:
alphas = np.logspace(-6, 2, 200)
lasso_cv = LassoCV(cv=5,random_state=17,alphas=alphas) # Ваш код здесь
lasso_cv.fit(X_train_scaled,y_train); # Ваш код здесь

In [50]:
lasso_cv.alpha_

0.0002833096101839324

**Выведите коэффициенты "лучшего" Lasso в порядке убывания влияния на качество вина. **<br>
**<font color='red'>Вопрос 3:</font> Какой признак "обнулился первым" в настроенной модели LASSO?**

In [52]:
lasso_cv_coef = pd.DataFrame(lasso_cv.coef_,X_train.columns) # Ваш код здесь
lasso_cv_coef.sort_values(by=0,key=abs,ascending=False) # Ваш код здесь
lasso_cv_predict_train = lasso_cv.predict(X_train_scaled)
lasso_cv_predict = lasso_cv.predict(X_holdout_scaled)

**Оцените среднеквадратичную ошибку модели на обучающей и тестовой выборках.**<br>
**<font color='red'>Вопрос 4:</font> Каковы среднеквадратичные ошибки настроенной LASSO-регрессии на обучающей и отложенной выборках?**

In [53]:
print("Mean squared error (train): %.3f" % mean_squared_error(y_true=y_train,y_pred=lasso_cv_predict_train))
print("Mean squared error (test): %.3f" % mean_squared_error(y_true=y_holdout,y_pred=lasso_cv_predict))

Mean squared error (train): 0.558
Mean squared error (test): 0.583


## Случайный лес

**Обучите случайный лес с параметрами "из коробки", фиксируя только random_state=17.**

In [57]:
forest = RandomForestRegressor(random_state=17)
forest.fit(X_train_scaled,y_train);
forest_pred_train = forest.predict(X_train_scaled)
forest_predict = forest.predict(X_holdout_scaled)

**<font color='red'>Вопрос 5:</font> Каковы среднеквадратичные ошибки случайного леса на обучающей выборке, на кросс-валидации (cross_val_score с scoring='neg_mean_squared_error' и остальными параметрами по умолчанию) и на отложенной выборке?**

In [61]:
print("Mean squared error (train): %.3f" % mean_squared_error(y_train,forest_pred_train))
print("Mean squared error (cv): %.3f" % cross_val_score(estimator=forest,X=X_train_scaled,y = y_train,scoring='neg_mean_squared_error').mean())
print("Mean squared error (test): %.3f" % mean_squared_error(y_holdout,forest_predict))

Mean squared error (train): 0.053
Mean squared error (cv): -0.414
Mean squared error (test): 0.372


**Настройте параметры min_samples_leaf и max_depth с помощью GridSearchCV и опять проверьте качество модели на кросс-валидации и на отложенной выборке.**

In [62]:
forest_params = {'max_depth': list(range(10, 25)), 
                 'min_samples_leaf': list(range(1, 8)),
                 'max_features': list(range(6,12))}

locally_best_forest = GridSearchCV(estimator=forest,param_grid=forest_params) # Ваш код здесь
locally_best_forest.fit(X_train_scaled,y_train) # Ваш код здесь

KeyboardInterrupt: 

In [63]:
locally_best_forest.best_params_, locally_best_forest.best_score_

AttributeError: 'GridSearchCV' object has no attribute 'best_params_'

**К сожалению, результаты  GridSearchCV не полностью воспроизводимы (могут отличаться на разных платформах даже при фиксировании *random_state*). Поэтому обучите лес с параметрами max_depth=19, max_features=7, и min_samples_leaf=1 (лучшие в моем случае).**<br>
**<font color='red'>Вопрос 6:</font> Каковы среднеквадратичные ошибки настроенного случайного леса на обучающей выборке, на кросс-валидации (cross_val_score с scoring='neg_mean_squared_error') и на отложенной выборке?**

In [65]:
best_forest = RandomForestRegressor(max_depth=19,max_features=7,min_samples_leaf=1)
best_forest.fit(X_train_scaled,y_train)
predict = best_forest.predict(X_holdout_scaled)

In [67]:
print("Mean squared error (cv): %.3f" % cross_val_score(estimator=forest,X=X_train_scaled,y = y_train,scoring='neg_mean_squared_error').mean())
print("Mean squared error (test): %.3f" % mean_squared_error(y_holdout,predict))

Mean squared error (cv): -0.414
Mean squared error (test): 0.370


**Оцените важность признаков с помощью случайного леса.**<br>
**<font color='red'>Вопрос 7:</font> Какой признак оказался главным в настроенной модели случайного леса?**

In [69]:
rf_importance = pd.DataFrame(best_forest.feature_importances_,X_train.columns)
rf_importance.sort_values(by=0,key=abs,ascending=False) # Ваш код здесь

,0
alcohol,0.209359
volatile acidity,0.119906
free sulfur dioxide,0.114102
density,0.092338
pH,0.072694
residual sugar,0.072506
total sulfur dioxide,0.069517
chlorides,0.069202
fixed acidity,0.063472
sulphates,0.058483


**Сделайте выводы о качестве моделей и оценках влияния признаков на качество вина с помощью этих трех моделей.**